In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-207032
Azure region: southcentralus
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-207032


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
cluster_name = "optimize-pipeline-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


#### 1. Training the model using Hyperdrive on our custom training script

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "C": choice(0.01, 0.001, 0.1, 1),
        "max_iter": choice(50,100),
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=10,
                             max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)

In [5]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show() 
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3d57884b-f346-4e29-9b27-9558bb50e3bc
Web View: https://ml.azure.com/runs/HD_3d57884b-f346-4e29-9b27-9558bb50e3bc?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-207032/workspaces/quick-starts-ws-207032&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-09-11T10:19:57.076743][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-09-11T10:19:57.7662611Z][SCHEDULER][INFO]Scheduling job, id='HD_3d57884b-f346-4e29-9b27-9558bb50e3bc_0' 
[2022-09-11T10:19:57.8863781Z][SCHEDULER][INFO]Scheduling job, id='HD_3d57884b-f346-4e29-9b27-9558bb50e3bc_1' 
[2022-09-11T10:19:58.0116804Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_3d57884b-f346-4e29-9b27-9558bb50e3bc_0' 
[2022-09-11T10:19:57.9924333Z][SCHEDULER][INFO]Scheduling job, id='HD_3d57884b-f346-4e29-9b27-9558bb50e3bc_2' 
[2022-09-11T10:19:58.046267][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to t

{'runId': 'HD_3d57884b-f346-4e29-9b27-9558bb50e3bc',
 'target': 'optimize-pipeline-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-11T10:19:56.410021Z',
 'endTimeUtc': '2022-09-11T10:37:35.766325Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'd7bffa5b-0c6c-4ae5-a503-40cafa909489',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': '8',
  'score': '0.9100509832483612',
  'best_child_run_id': 'HD_3d57884b-f346-4e29-9b27-9558bb50e3bc_5',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_3d57884b-f346-4e29-9b27-9558bb50e3bc_5'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues':

KeyError: 'log_files'

In [6]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Best Accuracy:', best_run_metrics['Accuracy'])

model = best_run.register_model(
    model_name="hyperdrive-best-model", model_path="."
)

Best Run Id:  HD_3d57884b-f346-4e29-9b27-9558bb50e3bc_5

 Best Accuracy: 0.9100509832483612


#### 2. Training the model using AutoML 

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset

# Create TabularDataset using TabularDatasetFactory
file_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = Dataset.Tabular.from_delimited_files(path=file_path)
 

In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=3)

In [9]:
# Submit your automl run
automl_run = exp.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_c97afae7-74de-440a-957e-0387cfd4f20c,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2022-09-11:11:11:41,485 INFO     [explanation_client.py:334] Using default datastore for uploads


In [11]:
# Retrieve and save your best automl model.

best_automl_run = automl_run.get_best_child()

best_automl_model_name = best_automl_run.properties['model_name']

best_automl_model = best_automl_run.register_model(model_name ="best-automl-model", model_path='.')